<h1><center><span style="font-family:Comic Sans MS">Scraping Data About Laptops On Jumia</span><center><h1>

<strong><span style="font-family:Lucida Sans">The aim of this project is to scrape all models of Laptops hosted for sale on the Jumia website. The data gotten from here would be saved in a "csv" format for easy analysis later</span><strong>

In [1]:
#Importing packages

from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np
from time import sleep
from time import time


In [2]:
#First we create sets of empty list to load our data into later 

description =[]
current_prices = []
ratings = []
old_prices = []
brands = []
discounts = []
models = []
stars =[]

# time() is the currnet time at any point of running the code
start_time = time()

# initial request is set to zero and increased by 1 on every successful request
request = 0

#Create a loop to iterate through the pages we would be scraping, in this case 50 pages
for i in range (1,51):
    
    link = "https://www.jumia.com.ng/laptops/?page={}#catalog-listing".format(i)
    page = requests.get(link)
    request += 1
#  displays each request so we that can monitor the process,not very necessary,you can comment it out.
    #print('request: {}'.format(request))
    
#We parse each request through BeautifulSoup 
    pages = soup(page.text , "html.parser")
    
#We find the class that contains all the information we want to scrape after inspecting the webpage
    containers = pages.find_all('article', class_="prd _fb col c-prd")
#We iterate through every item we want to get and then add it to the empty list we created earlier.
    for container in containers:
        
        #Brands
        brand = container.find('h3', class_='name').text.split(' ')[0]
        brands.append(brand)
        
        #Phone models
        phone_model = container.find('h3', class_='name').text.split(' ')[:3]
        model= ' '.join(phone_model)
        models.append(model)
        
        #Phone Descriptions
        descrip = container.find('h3', class_='name').text.split(' ')[:]
        descriptions= ' '.join(descrip)
        description.append(descriptions)
        
        #Current Prices
        cprice = container.find('div', class_='prc').text.split(' ')[:]
        current_price= ' '.join(cprice)
        current_prices.append(current_price)

#Some products have some value applicable e.g Discount, hence we create an exception that would append "None" if no value was found
        
        #Old Prices
        try:
            oldprice= container.find('div', class_='old').text.split(' ')[:]
            old_price= ' '.join(oldprice)
            old_prices.append(old_price)
        except Exception:
            oldprice = None
            old_prices.append(oldprice)
            
        #Discount
        try:
            dscount = container.find('div', class_='tag _dsct _sm').text.split(' ')[:]
            discount= ' '.join(dscount)
            discounts.append(discount)
        except Exception:
            dscount = None
            discounts.append(dscount)
        
        #Ratings
        try:
            rate = container.find('div', class_='rev').text.replace("(","").replace(")","").replace("5"," ").replace(" "," ")[10:]
            rating= rate.strip()
            ratings.append(rating)
        except Exception:
            rate = None
            ratings.append(rate)
            
        #Number of Stars
        try:

            no_star = container.find('div', class_='rev').text.split(' ')[:1]
            star= ' '.join(no_star)
            stars.append(star)
        except Exception:
            no_star = None
            stars.append(no_star)
            
#We then create a DataFrame to store all the items and values.
Laptops = pd.DataFrame({'Brand': brands, 'Laptop Model': models,'Old Price': old_prices,'Discounts': discounts,
                                 'description': description,'Ratings': ratings,'Stars': stars, 'Current Prices':current_prices}, columns =[
        'Brand','Laptop Model','Current Prices','Old Price','Discounts','description','Ratings','Stars'])
elasped_time = time() - start_time
print("Completed in {}secs".format(elasped_time))

Completed in 61.6786584854126secs


<span style="font-family:Lucida Sans">We can do a quick view of the Data</span>

In [3]:
#We were able to scrape a total of 2000 items
Laptops.shape

(2000, 8)

In [4]:
#A quick view of the first 10 rows
Laptops.head(10)


,Brand,Laptop Model,Current Prices,Old Price,Discounts,description,Ratings,Stars
0,Hp,Hp STREAM 11,"₦ 130,150","₦ 188,000",31%,Hp STREAM 11 INTEL CELERON® 4GB RAM 32GB EMMC ...,1,4
1,Mtk,Mtk 10.1 Inch,"₦ 54,999","₦ 60,000",8%,Mtk 10.1 Inch Android 6.0 Netbook A33 Mini Laptop,4,2.8
2,Hp,Hp Stream 11,"₦ 137,000","₦ 146,500",6%,Hp Stream 11 Intel Celeron D/C 4GB RAM- 32GB ...,None,None
3,Hp,Hp Stream 11,"₦ 105,000","₦ 120,000",13%,Hp Stream 11 Intel Celeron 2GB RAM- 32GB HDD W...,None,None
4,TECLAST,TECLAST F15 Laptop,"₦ 170,345","₦ 379,980",55%,TECLAST F15 Laptop 8GB RAM 256 RAM SSD Intel UHD,1,1
5,Hp,Hp Stream 11,"₦ 130,149","₦ 146,500",11%,Hp Stream 11 Intel Celeron D/C 4GB RAM- 32GB ...,1,5
6,Hp,Hp Stream 11,"₦ 105,000","₦ 125,000",16%,Hp Stream 11 Intel Celeron 2GB RAM- 32GB HDD W...,None,None
7,Hp,Hp (15) 15.6-Inch,"₦ 232,590","₦ 240,000",3%,Hp (15) 15.6-Inch Intel Core -i3 2.0GHz 8GB R...,1,5
8,Hp,Hp Notebook -,"₦ 160,000",None,None,Hp Notebook - 15- Intel Pentium N3710 Quad Cor...,None,None
9,Lenovo,Lenovo Ideapad 8th,"₦ 198,500","₦ 225,000",12%,Lenovo Ideapad 8th Gen Intel Core I3 (4GB/1TB)...,2,2.5


In [5]:
#A quick view of the last 10 rows
Laptops.tail(10)


,Brand,Laptop Model,Current Prices,Old Price,Discounts,description,Ratings,Stars
1990,Hp,Hp X360 14,"₦ 370,000","₦ 390,000",5%,Hp X360 14 Intel Core I5 1TB HDD 12GB RAM Touc...,None,None
1991,Hp,Hp Stream 14,"₦ 170,000",None,None,Hp Stream 14 Intel Celeron D/C 64GB HDD- 4GB ...,None,None
1992,Hp,Hp Notebook 15,"₦ 215,000","₦ 230,000",7%,Hp Notebook 15 Intel Pentium 4GB RAM 500 HDD W...,None,None
1993,Hp,Hp Notebook 15,"₦ 223,000","₦ 230,000",3%,"Hp Notebook 15 -Intel Core I3, 2.4Ghz, 1TB HDD...",None,None
1994,Hp,Hp Notebook -,"₦ 230,000",None,None,Hp Notebook - 15-bs061nia - Intel® Core™ I3 (2...,None,None
1995,Hp,Hp 15 Intel,"₦ 223,000","₦ 230,000",3%,Hp 15 Intel Core I3 2.4Ghz 1TB HDD 4GB RAM+ 32...,2,4
1996,Hp,Hp Notebook 15,"₦ 223,000","₦ 230,000",3%,"Hp Notebook 15 -Intel Core I3, 2.4Ghz, 1TB HDD...",None,None
1997,Hp,Hp Notebook 15,"₦ 223,000","₦ 230,000",3%,"Hp Notebook 15 -Intel Core I3, 2.4Ghz, 1TB HDD...",None,None
1998,Hp,Hp Pavilion 15,"₦ 345,000",None,None,"Hp Pavilion 15 Intel Core I5 2.5GHz (8GB RAM, ...",None,None
1999,Hp,Hp Notebook 15,"₦ 225,500",None,None,"Hp Notebook 15 Intel Corei3 1tb HDD, 4GB RAM, ...",None,None


In [6]:
Laptops.describe()

,Brand,Laptop Model,Current Prices,Old Price,Discounts,description,Ratings,Stars
count,2000,2000,2000,944,944,2000,79,79
unique,68,506,549,229,36,1252,7,14
top,Hp,Hp Notebook 15,"₦ 105,000","₦ 120,000",13%,Hp Stream 11 Intel Celeron 2GB RAM- 32GB HDD W...,1,5
freq,1379,220,92,73,103,80,53,25


In [7]:
Laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Brand           2000 non-null   object
 1   Laptop Model    2000 non-null   object
 2   Current Prices  2000 non-null   object
 3   Old Price       944 non-null    object
 4   Discounts       944 non-null    object
 5   description     2000 non-null   object
 6   Ratings         79 non-null     object
 7   Stars           79 non-null     object
dtypes: object(8)
memory usage: 125.1+ KB


In [8]:
#Saving the data as a CSV file
Laptops.to_csv("Laptops.csv", index =False)

<br>

<h2><center><span style="font-family:Comic Sans MS">Thank You For Viewing This Notebook</span><center><h2>